# Tupro3 PKB (Learning)

Machine Learning

Diberi data traintest.xlsx, ditugaskan untuk menggunakan data ini untuk memprediksi masalah klasifikasi biner

Langkah-langkah :
- Membaca data
- Membuat metode learning
- Latih model data
- Menguji dan mengevaluasi model data

# 1. Membaca data

In [39]:
import pandas as pd
from math import sqrt
from statistics import mode

In [40]:
data = pd.read_excel('traintest.xlsx', index_col='id')
test_data = pd.read_excel('traintest.xlsx', sheet_name='test', index_col='id')[['x1', 'x2', 'x3']]

print('Banyak baris pada train :', len(data))
print('Banyak baris pada test :', len(test_data))

Banyak baris pada train : 296
Banyak baris pada test : 10


File traintest.xlsx memiliki 296 baris data pada datasheet yang bernama train, dan 10 baris data pada datasheet yang bernama test. Pertama-tama, mari kita ambil 20% dari datasheet train secara acak untuk digunakan sebagai testing akurasi dan 80% sisanya untuk training data.

In [41]:
def ambilsampel(data):
    acctest_data = data.sample(frac=.2)
    train_data = data.drop(acctest_data.index)
    return acctest_data, train_data

acctest_data, train_data = ambilsampel(data)
#print('Banyak baris pada train_data :', len(train_data))
#print('Banyak baris pada acctest_data :', len(acctest_data))

In [42]:
train_data.equals(acctest_data)

False

Lalu untuk mempermudah dilakukannya testing akurasi, kita bagi data training menjadi 2 dimana salah satunya menyimpan nilai x dan yang lainnya menyimpan nilai y.

In [43]:
def splitdata(acctest_data = None, train_data = None):
    
    if train_data is not None and acctest_data is None:
        train_data_x = train_data[['x1', 'x2', 'x3']].values.tolist()
        train_data_y = train_data['y'].values.tolist()
        return train_data_x, train_data_y

    elif train_data is None and acctest_data is not None:
        acctest_data_x = acctest_data[['x1', 'x2', 'x3']].values.tolist()
        acctest_data_y = acctest_data['y'].values.tolist()
        return acctest_data_x, acctest_data_y
    
    else:
        train_data_x = train_data[['x1', 'x2', 'x3']].values.tolist()
        train_data_y = train_data['y'].values.tolist()
        acctest_data_x = acctest_data[['x1', 'x2', 'x3']].values.tolist()
        acctest_data_y = acctest_data['y'].values.tolist()
        return train_data_x, train_data_y, acctest_data_x, acctest_data_y


# 2. Membuat metode learning

Ada berbagai macam metode yang bisa digunakan untuk memecahkan masalah binary classification. Diantara metode-metode yang ada, pada kali ini kita akan mencoba metode KNN (K Nearest Neighbour).

Metode KNN atau yang biasa disebut dengan Lazy Learning adalah sebuah metode yang bekerja dengan cara membandingkan jarak suatu titik yang ditentukan oleh titik baru dengan titik-titik lain yang sudah ditentukan keanggotaannya. Kemudian, ditentukan keanggotaan titik baru ini berdasarkan keanggotaan K titik yang terdekat dengan titik baru.

Untuk membuat metode KNN, diperlukan :
- Perhitungan jarak antar titik (akan digunakan euclidean distance)
- Pengklasifikasian berdasarkan K titik terdekat

Maka dari itu dibuat 3 komponen fungsi, yang masing-masing memiliki tugas diantaranya :
- eucl_dist = Menghitung jarak antar titik (titik baru dengan titik yang dimasukan)
- neighbours = Mengembalikan klasifikasi dari titik baru berdasarkan perbandingan jarak ke K neighbour yang ada
- prediction = Mengembalikan array berisi prediksi yang didapatkan dari melakukan perulangan pada fungsi neighbours untuk setiap titik baru yang ada pada dataset

In [44]:
def eucl_dist(x1, x2):
    distance = 0
    for i in range(3):
        distance += pow((x2[i]-x1[i]), 2)
    return sqrt(distance)

In [45]:
def neighbours(train_data_x, train_data_y, titik_baru, k):
    
    dist_list = []
    i = 0
    while i != len(train_data_x):
        #mengembalikan jarak titik baru dengan titik pada train data dan juga klasifikasi dari train datanya
        dist_list.append([eucl_dist(train_data_x[i], titik_baru), train_data_y[i]])
        i += 1
    dist_list.sort(key=lambda x: x[0])
    dist_list_final = dist_list[:k]
    
    j = 0
    #klasifikasi
    vote_0 = 0
    vote_1 = 0
    
    while j < len(dist_list_final):
        val = dist_list_final[j][1]
        if val == 0:
            vote_0 += 1
        elif val == 1:
            vote_1 += 1
        j += 1
        
    #jika 0 > 1
    if vote_0 > vote_1:
        return 0
    #jika 1 > 0
    elif vote_0 < vote_1:
        return 1
    #jika 0 = 1
    else:
        count_0 = 0
        count_1 = 0
        for i in train_data_y:
            if i == 1:
                count_1 += 1
            else:
                count_0 += 1
        if count_1 > count_0:
            return 1
        else:
            return 0

In [46]:
#jika ans False (tidak diketahui jawabannya), maka hanya mereturn prediksinya saja
def prediction(k, train_data_x, train_data_y, acctest_data_x, ans=None):
    #prediction banding actual
    predict = []
    count = 0
    while count != len(acctest_data_x):
        if ans is None:
            predict.append(neighbours(train_data_x, train_data_y, acctest_data_x[count], k))
        else:
            predict.append([neighbours(train_data_x, train_data_y, acctest_data_x[count], k), ans[count]])
        count += 1
    return predict

In [47]:
#testing fungsi prediction
train_data_x, train_data_y, acctest_data_x, acctest_data_y = splitdata(acctest_data, train_data)
prediction(2, train_data_x, train_data_y, acctest_data_x, acctest_data_y)

[[0, 1],
 [1, 1],
 [1, 0],
 [1, 0],
 [1, 0],
 [0, 1],
 [1, 0],
 [1, 1],
 [1, 0],
 [1, 0],
 [1, 0],
 [1, 0],
 [1, 1],
 [1, 1],
 [1, 1],
 [1, 1],
 [1, 1],
 [1, 1],
 [1, 1],
 [1, 1],
 [1, 1],
 [1, 1],
 [0, 0],
 [1, 1],
 [1, 1],
 [1, 1],
 [1, 1],
 [1, 0],
 [1, 0],
 [0, 0],
 [1, 1],
 [1, 1],
 [1, 1],
 [0, 0],
 [1, 0],
 [1, 0],
 [1, 1],
 [1, 0],
 [0, 1],
 [1, 1],
 [1, 1],
 [1, 1],
 [1, 0],
 [1, 1],
 [1, 1],
 [1, 1],
 [1, 1],
 [1, 0],
 [1, 1],
 [1, 1],
 [1, 1],
 [1, 1],
 [0, 1],
 [1, 1],
 [1, 0],
 [1, 0],
 [1, 0],
 [1, 1],
 [1, 1]]

# 3. Latih model data

Setelah metode learning dibuat, langkah selanjutnya adalah menguji akurasi menggunakan dataset yang sudah dibagi-bagi sebelumnya. Adapun tujuan dari menguji akurasi ini adalah untuk mendapatkan nilai K terbaik yang pada nantinya akan digunakan untuk ujicoba real data (test_data).

Untuk menguji akurasi data, pada kali ini akan digunakan confusion matrix.

In [48]:
#accuracy test menggunakan confusion matrix
def accutest(train_data, acctest_data):
    train_data_x, train_data_y, acctest_data_x, acctest_data_y = splitdata(acctest_data, train_data)
    nilai_k = range(1, 11, 1)
    acc_res = []
    for k in nilai_k:
        TP = 0
        TN = 0
        FP = 0
        FN = 0
        test_arr = prediction(k, train_data_x, train_data_y, acctest_data_x, acctest_data_y)
        i = 0
        while i != len(test_arr):
            if test_arr[i][0] == 1 and test_arr[i][1] == 1:
                TP += 1
            elif test_arr[i][0] == 1 and test_arr[i][1] == 0:
                FP += 1
            elif test_arr[i][0] == 0 and test_arr[i][1] == 1:
                FN += 1
            elif test_arr[i][0] == 0 and test_arr[i][1] == 0:
                TN += 1
            i += 1
        accuracy = (TP+TN)/(TP+TN+FP+FN)
        precision = TP/(TP+FP)
        recall = TP/(TP+FN)
        f1_score = (2*precision*recall)/(precision+recall)
        acc_res.append([k, accuracy, f1_score])
    acc_res.sort(key=lambda x: (-x[1], x[2]))
    return acc_res[0]

In [49]:
accutest(train_data, acctest_data)

[7, 0.6610169491525424, 0.782608695652174]

Jika diamati, akurasi akan selalu berganti-ganti seiring dengan semua fungsi dijalankan. Oleh karena itu, akan dibuatkan fungsi yang selalu mengulang pengambilan sampel dan menguji akurasi dari nilai k yang ada sampai menemukan nilai k yang sering muncul. Banyaknya perulangan yang dilakukan adalah sebanyak 50 kali.

In [50]:
def accutest_loop(data):
    most_k_val = []
    print("Mencari nilai k terbaik ...\n")
    i = 0
    while i != 50:
        acctest_data, train_data = ambilsampel(data)
        k_val, accuracy, fscore = accutest(train_data, acctest_data)
        most_k_val.append(k_val)
        print(k_val, accuracy*100)
        i+=1
    best_k = mode(most_k_val)
    print("\nNilai K yang digunakan adalah", best_k)
    return best_k

In [51]:
accutest_loop(data)

Mencari nilai k terbaik ...

1 76.27118644067797
1 74.57627118644068
10 71.1864406779661
9 81.35593220338984
5 77.96610169491525
4 74.57627118644068
9 77.96610169491525
2 81.35593220338984
9 77.96610169491525
1 69.49152542372882
8 69.49152542372882
10 74.57627118644068
10 81.35593220338984
1 83.05084745762711
5 79.66101694915254
9 77.96610169491525
9 86.4406779661017
5 71.1864406779661
4 88.13559322033898
1 74.57627118644068
9 71.1864406779661
9 76.27118644067797
1 76.27118644067797
8 74.57627118644068
3 81.35593220338984
4 84.7457627118644
4 72.88135593220339
8 79.66101694915254
9 81.35593220338984
2 81.35593220338984
9 76.27118644067797
9 79.66101694915254
5 79.66101694915254
9 74.57627118644068
2 83.05084745762711
10 74.57627118644068
5 77.96610169491525
2 83.05084745762711
1 69.49152542372882
8 81.35593220338984
1 71.1864406779661
8 76.27118644067797
2 69.49152542372882
4 76.27118644067797
10 86.4406779661017
2 79.66101694915254
2 79.66101694915254
3 79.66101694915254
7 74.57627118

9

# 4. Pengujian model data

Setelah dibuatnya metode learning, latih data, dan fungsi yang mencari nilai k yang optimal, maka akan dicoba model ini untuk real data (test_data). Hasil akan disimpan kedalam file excel 'HasilML.xlsx'

Kita akan menggunakan semua dataset (termasuk dataset yang belum dibagi-bagi)

In [52]:
def main(data, test_data, train_data, acctest_data):
    
    data_x = data[['x1', 'x2', 'x3']].values.tolist()
    data_y = data['y'].values.tolist()
    test = test_data.values.tolist()
    
    #cari nilai k terbaik
    best_k_val = accutest_loop(data)
    #lakukan prediksi
    print("Berikut hasil prediksi test data :\n")
    result = prediction(best_k_val, data_x, data_y, test)
    dfres = test_data
    dfres['y'] = result
    return dfres

In [53]:
datafinal = main(data, test_data, train_data, acctest_data)
datafinal

Mencari nilai k terbaik ...

3 79.66101694915254
2 81.35593220338984
1 81.35593220338984
5 79.66101694915254
1 77.96610169491525
2 79.66101694915254
1 72.88135593220339
9 83.05084745762711
2 72.88135593220339
9 83.05084745762711
3 69.49152542372882
5 72.88135593220339
1 71.1864406779661
3 79.66101694915254
6 77.96610169491525
10 76.27118644067797
5 77.96610169491525
7 76.27118644067797
9 77.96610169491525
2 77.96610169491525
10 79.66101694915254
1 74.57627118644068
9 81.35593220338984
2 79.66101694915254
10 69.49152542372882
9 81.35593220338984
7 77.96610169491525
7 81.35593220338984
9 76.27118644067797
3 71.1864406779661
1 77.96610169491525
1 76.27118644067797
9 71.1864406779661
10 77.96610169491525
9 76.27118644067797
9 72.88135593220339
10 72.88135593220339
2 76.27118644067797
10 74.57627118644068
7 74.57627118644068
10 66.10169491525424
2 84.7457627118644
10 79.66101694915254
9 79.66101694915254
9 77.96610169491525
9 74.57627118644068
1 77.96610169491525
7 81.35593220338984
1 79.66

,x1,x2,x3,y
id,,,,
297,43,59,2,1
298,67,66,0,1
299,58,60,3,1
300,49,63,3,1
301,45,60,0,1
302,54,58,1,1
303,56,66,3,1
304,42,69,1,1
305,50,59,2,1


In [54]:
datafinal.to_excel('HasilML.xlsx', sheet_name='Hasil Akhir')

PermissionError: [Errno 13] Permission denied: 'HasilML.xlsx'

# Scrap function (Debugging Purposes Only)

In [ ]:
len(range(1, 60, 2))

In [ ]:
#buat fungsi menghitung akurasi, nanti buat jg sistem yang bisa ngecycle k biar dapet nilai k yg terbaik
def accuracy_test(train_data_x, train_data_y, acctest_data_x, acctest_data_y):
    nilai_k = range(1, 60, 2)
    acc_res = []
    for k in nilai_k:
        correctans = 0
        test_arr = prediction(k, train_data_x, train_data_y, acctest_data_x, acctest_data_y)
        i = 0
        while i != len(test_arr):
            if test_arr[i][0] == test_arr[i][1]:
                correctans += 1
            i += 1
        avg_acc = (correctans/len(test_arr))*100
        acc_res.append([k, avg_acc])
    acc_res.sort(key=lambda x: x[1], reverse=True)
    return acc_res

In [ ]:
test_result = accuracy_test(train_data_x, train_data_y, acctest_data_x, acctest_data_y)
test_result
#print("Didapatkan akurasi terbaik dengan k", test_result[0], "dan akurasi", test_result[1], "%")